### 1. Set-up

#### 1.1 Imports

In [ ]:
# Imports
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import math
import time
import numpy as np
import sys
import argparse
import os
import matplotlib.pyplot as plt
import pandas as pd
import math
import seaborn as sns

#### 1.2 CUDA

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### 2. Data Pre-Processing

#### 2.1 Helper Functions

In [ ]:
# Encoding and decoding
def decode(vocab,corpus):
    
    text = ''
    for i in range(len(corpus)):
        wID = corpus[i]
        text = text + vocab[wID] + ' '
    return(text)

def encode(words,text):
    corpus = []
    tokens = text.split(' ')
    for t in tokens:
        try:
            wID = words[t][0]
        except:
            wID = words['<unk>'][0]
        corpus.append(wID)
    return(corpus)

def read_encode(file_name,vocab,words,corpus,threshold):
    
    wID = len(vocab)
    
    if threshold > -1:
        with open(file_name,'rt', encoding='utf8') as f:
            for line in f:
                line = line.replace('\n','')
                tokens = line.split(' ')
                for t in tokens:
                    try:
                        elem = words[t]
                    except:
                        elem = [wID,0]
                        vocab.append(t)
                        wID = wID + 1
                    elem[1] = elem[1] + 1
                    words[t] = elem

        temp = words
        words = {}
        vocab = []
        wID = 0
        words['<unk>'] = [wID,100]
        vocab.append('<unk>')
        for t in temp:
            if temp[t][1] >= threshold:
                vocab.append(t)
                wID = wID + 1
                words[t] = [wID,temp[t][1]]
            
                    
    with open(file_name,'rt', encoding='utf8') as f:
        for line in f:
            line = line.replace('\n','')
            tokens = line.split(' ')
            for t in tokens:
                try:
                    wID = words[t][0]
                except:
                    wID = words['<unk>'][0]
                corpus.append(wID)
                
    return [vocab,words,corpus]

def plot_data(x, y1, y2, xlabel, ylabel, title, color1, color2, label1, label2):
    plt.plot(x, y1, color1, label=label1)
    plt.plot(x, y2, color2, label=label2)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.show()

#### 2.2 Create ngrams

In [ ]:
def split_bios(data, words):
  fake_val, real_val = words["[FAKE]"][0], words["[REAL]"][0]
  fake_list = (data == fake_val)
  real_list = (data == real_val)
  target_indices = (fake_list + real_list).nonzero()
  bios_list = []
  start_i = 0

  for end_i in target_indices:
    bio = data[start_i:end_i+1]
    bios_list.append(bio)

    start_i = end_i + 1

  return bios_list

def clean_bios(bios, words):
  for bio_i in range(len(bios)):
    # Remove new lines and punctuation
    new_line_idx = words[''][0]
    comma_idx = words[','][0]
    period_idx = words['.'][0]
    colon_idx = words[':'][0]
    semicolon_idx = words[';'][0]

    punctuation_indices = [new_line_idx, comma_idx, period_idx, colon_idx, semicolon_idx]

    bios[bio_i] = list(filter(lambda x: x not in punctuation_indices, bios[bio_i]))

  return bios

def create_ngrams(bios, words, ngram_size):
  ngram_list = []

  for bio_i, bio in enumerate(bios):
    bio_len = len(bio)
    i = 0
    
    while i < bio_len - ngram_size:
      start, stop = i, i+ngram_size
      context = torch.tensor(bio[start:stop]).to(device)
      label = torch.tensor(bio[stop]).to(device)

      ngram = [context, label]
      ngram_list.append(ngram)

      i += 1

  return ngram_list


#### 2.3 Data Loader

In [ ]:
class NgramDataset(Dataset):
    def __init__(self, ngrams_data):
        self.ngrams_data_df = pd.DataFrame(ngrams_data, columns=("context", "label"))
        self.context = self.ngrams_data_df["context"]
        self.label = self.ngrams_data_df["label"]

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        return self.context[idx], self.label[idx]

### 3. Model

In [ ]:
# FeedForward Model
class FFNN(nn.Module):
    def __init__(self, vocab, words, d_model, d_hidden, dropout, ngram_size):
        super().__init__() 
    
        # Class parameters
        self.vocab = vocab
        self.words = words
        self.vocab_size = len(self.vocab)
        self.d_model = d_model
        self.d_hidden = d_hidden
        self.ngram_size = ngram_size
        
        # Dropout
        self.dropout = nn.Dropout(p=dropout)

        # Embedding Layer
        self.input_embedding = nn.Embedding(self.vocab_size, self.d_model)

        # Linear Layers
        self.fc1 = nn.Linear(ngram_size * d_model, self.d_hidden)
        self.output_embedding = nn.Linear(self.d_hidden, self.vocab_size)

        # Nonlinear Layer
        self.activation = nn.ReLU()

        # Setting weights
        self.init_weights()
                
    # Initialize weights for foward layer
    def init_weights(self):
        weight_range = 0.1
        
        self.input_embedding.weight.data.uniform_(-weight_range, weight_range)
        self.fc1.weight.data.uniform_(-weight_range, weight_range)
        self.fc1.bias.data.zero_()

    # Forward
    def forward(self, src):
        # Embeddings are fed into the forward layer
        embeds = self.input_embedding(src).view(-1, self.d_model * self.ngram_size)
        x = self.dropout(self.activation(self.fc1(embeds)))
        x = self.output_embedding(x).view(-1, self.vocab_size)
        x = F.log_softmax(x, dim=1)
        return x

### 4. Train

#### 4.1 Metrics

#### 4.2 Train One Epoch

In [ ]:
def train_one_epoch(model, optimizer, criterion, scheduler, train_dataloader, valid_dataloader):

  model.train(True)
  # Training Set
  running_train_acc, running_train_loss = 0, 0
  train_num_batches = len(train_dataloader)
  
  for batch_idx, (train_contexts, train_labels) in enumerate(train_dataloader):
      model.zero_grad()

      train_logits = model(train_contexts)
      train_loss = criterion(train_logits, train_labels)
      
      running_train_loss += train_loss.item()

      # if batch_idx % 10000 == 0: 
      #   print(f"Batch: {batch_idx+1}")
      #   print(f"Loss: {running_train_loss / (batch_idx+1)}")
      #   print(f"Accuracy: {running_train_acc / (batch_idx+1)}\n")

      train_loss.backward()
      optimizer.step()

      train_preds = torch.argmax(train_logits, dim=1)
      train_acc = accuracy_score(np.array(train_labels.cpu()), np.array(train_preds.cpu()))
      running_train_acc += train_acc

  train_av_acc = running_train_acc / train_num_batches
  train_av_loss = running_train_loss / train_num_batches

  model.train(False)

  # Validation Set
  running_valid_acc, running_valid_loss = 0, 0
  valid_num_batches = len(valid_dataloader)
  for batch_idx, (valid_contexts, valid_labels) in enumerate(valid_dataloader):
    
    valid_logits = model(valid_contexts)
    valid_loss = criterion(valid_logits, valid_labels)
    running_valid_loss += valid_loss.item()

    valid_preds = torch.argmax(valid_logits, dim=1)
    valid_acc = accuracy_score(np.array(valid_labels.cpu()), np.array(valid_preds.cpu()))
    running_valid_acc += valid_acc
    

  valid_av_acc = running_valid_acc / valid_num_batches
  valid_av_loss = running_valid_loss / valid_num_batches

  scheduler.step(math.exp(valid_av_loss))

  return train_av_acc, train_av_loss, valid_av_acc, valid_av_loss


#### 4.3 Train Multiple Epochs

In [ ]:
def train_loop(model, optimizer, criterion, scheduler, train_dataloader, valid_dataloader, epochs):
  train_accuracies = []
  train_losses = []
  valid_accuracies = []
  valid_losses = []
  epochs_list = list(range(epochs))

  for i in epochs_list:
    print(f"Epoch: {i+1} /////////////////////////////////////")
    # model.train(True)
    train_av_acc, train_av_loss, valid_av_acc, valid_av_loss = train_one_epoch(model, optimizer, criterion, scheduler, train_dataloader, valid_dataloader)
    # model.train(False)

    train_perplexity = math.exp(train_av_loss)
    valid_perplexity = math.exp(valid_av_loss)

    print(f"Train Accuracy: {train_av_acc}")
    print(f"Train Loss: {train_av_loss}")
    print(f"Train Perplexity: {train_perplexity}\n")

    print(f"Valid Accuracy: {valid_av_acc}")
    print(f"Valid Loss: {valid_av_loss}")
    print(f"Valid Perplexity: {valid_perplexity}\n")

    train_accuracies.append(train_av_acc)
    train_losses.append(train_av_loss)
    valid_accuracies.append(valid_av_acc)
    valid_losses.append(valid_av_loss)

  print(train_accuracies)
  print(train_losses)
  print(valid_accuracies)
  print(valid_losses)
  print(epochs_list)
  # Graph Accuracies and Loss
  plot_data(x=epochs_list, y1=train_accuracies, y2=valid_accuracies, xlabel="Epochs", ylabel="Accuracy", title="Accuracy", color1='r', color2='b', label1='Train', label2='Valid')
  plot_data(x=epochs_list, y1=train_losses, y2=valid_losses, xlabel="Epochs", ylabel="Loss", title="Loss", color1='r', color2='b', label1='Train', label2='Valid')

### 5. Validation

### 6. Test

In [ ]:
def create_histogram(model, dataloader, ngram_size, words):
  bio_len = 1
  fake_bio_probs, real_bio_probs = [], []
  total_bio_prob = 0
  MAX_VAL = 25000

  fake_val, real_val = words["[FAKE]"][0], words["[REAL]"][0]  

  for batch_idx, (context, label) in enumerate(dataloader):
    probs = model(context).squeeze()
    # probs = F.softmax(probs)
    label_prob = probs[label]

    if batch_idx % 50000 == 0: 
      print(f"{batch_idx}/{len(dataloader)}")
      print(label_prob)
    if label_prob == 0:
      label_prob += 0.00000001

    # pseudo_prob = -math.log(label_prob)
    pseudo_prob = -label_prob
    total_bio_prob += pseudo_prob

    if label == fake_val:
      bio_prob = total_bio_prob / bio_len
      if bio_prob < MAX_VAL: fake_bio_probs.append(bio_prob)
      bio_len = 1
      total_bio_probs = 0
    elif label == real_val:
      bio_prob = total_bio_prob / bio_len
      if bio_prob < MAX_VAL: real_bio_probs.append(bio_prob)
      bio_len = 1
      total_bio_probs = 0
    else:
      bio_len += 1

  sns.set(style="darkgrid")
 
  print(fake_bio_probs)
  print(real_bio_probs)

  plt.figure(0)
  sns.kdeplot(np.array(fake_bio_probs))
  sns.kdeplot(np.array(real_bio_probs))
  plt.xlabel("Negative Average Log Probs")
  plt.ylabel("Density")
  plt.show()

  plt.figure(1)
  plt.hist(fake_bio_probs, label='Fakes', alpha=0.5, bins=100)
  plt.hist(real_bio_probs, label='Reals', alpha=0.5, bins=100)
  plt.legend(loc='upper left')
  plt.xlabel("Negative Average Log Probs")
  plt.ylabel("Frequency")
  plt.show()

In [ ]:
def ffnn_test_model(model, dataloader, words, threshold):
  bio_len = 1
  truth, preds = [], []
  total_bio_prob = 0

  fake_val, real_val = words["[FAKE]"][0], words["[REAL]"][0]  

  for batch_idx, (context, label) in enumerate(dataloader):
    probs = model(context).squeeze()
    # probs = F.softmax(probs)
    label_prob = probs[label]

    if label_prob == 0:
      label_prob += 0.00000001

    # pseudo_prob = -math.log(label_prob)
    pseudo_prob = -label_prob
    total_bio_prob += pseudo_prob

    if label == fake_val or label == real_val:
      truth.append(label.item())

      bio_prob = total_bio_prob / bio_len
      if bio_prob < threshold:
        preds.append(real_val)
      else:
        preds.append(fake_val)

      bio_len = 1
      total_bio_probs = 0
    else:
      bio_len += 1

  acc = accuracy_score(np.array(truth), np.array(preds))
  confusion_mat = confusion_matrix(truth, preds)
  disp = ConfusionMatrixDisplay(confusion_matrix=confusion_mat, display_labels=["REAL", "FAKE"])
  disp.plot()
  plt.show()
  print(f"Test Accuracy: {acc}")
    

### 7. Running Code


#### 7.1 Loading Parameters

In [ ]:
class Params:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)
            
model_map = {0: 'FFNN', 1: 'LSTM', 2: 'FFNN_CLASSIFY', 3: 'LSTM_CLASSIFY'}
train_map = {0: 'data/real.train.tok', 1: 'data/fake.train.tok', 2: 'data/mix.train.tok'}
valid_map = {0: 'data/real.valid.tok', 1: 'data/fake.valid.tok', 2: 'data/mix.valid.tok'}
test_map = {0: 'data/real.test.tok', 1: 'data/fake.test.tok', 2: 'data/mix.test.tok', 3: 'data/blind.test.tok'}

model_type = model_map[0]

# Types of data
train_type = train_map[2]
valid_type = valid_map[2]
test_type = test_map[2]

args = {
    "d_model": 16,
    "d_hidden": 64,
    "n_layers": 3,
    "batch_size": 20,
    "seq_len": 30,
    "printevery": 5000,
    "window": 3,
    "epochs": 20,
    "lr": 0.0001,
    "dropout": 0.35,
    "clip": 2.0,
    "model": model_type,
    "savename": model_type.lower(),
    "loadname": model_type.lower(),
    "trainname": train_type,
    "validname": valid_type,
    "testname": test_type
}

#### 7.2 Main

In [ ]:
# Main Function
def main(args): 
    torch.manual_seed(0)
    
    # params
    params = Params(**args)
    train_name = params.trainname
    valid_name = params.validname
    test_name = params.testname
    model_type = params.model
    d_model = params.d_model
    d_hidden = params.d_hidden
    dropout = params.dropout
    epochs = params.epochs
    window = params.window
    batch_size = params.batch_size
    lr = params.lr

    # Extract vocab and words
    [train_vocab,train_words,train] = read_encode(train_name,[],{},[],3)
    train_data = torch.tensor(train)

    [valid_vocab,valid_words,valid] = read_encode(valid_name,train_vocab,train_words,[],-1)
    valid_data = torch.tensor(valid)

    [test_vocab,test_words,test] = read_encode(test_name,train_vocab,train_words,[],-1)
    test_data = torch.tensor(test)
    
    if model_type == 'FFNN':

      # Process Train Data
      train_bios = split_bios(train_data, train_words)
      train_bios = clean_bios(train_bios, train_words)
      train_ngrams_data = create_ngrams(train_bios, train_words, window)

      train_ngram_dataset = NgramDataset(train_ngrams_data)
      train_ngram_dataloader = DataLoader(train_ngram_dataset, batch_size=batch_size)

      # Process Valid Data
      valid_bios = split_bios(valid_data, valid_words)
      valid_bios = clean_bios(valid_bios, valid_words)
      valid_ngrams_data = create_ngrams(valid_bios, valid_words, window)

      valid_ngram_dataset = NgramDataset(valid_ngrams_data)
      valid_ngram_dataloader = DataLoader(valid_ngram_dataset, batch_size=batch_size)

      ngram_model = FFNN(train_vocab, train_words, d_model, d_hidden, dropout, window).to(device)
      optimizer = torch.optim.Adam(ngram_model.parameters(), lr=lr)
      criterion = nn.NLLLoss()
      scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=0)

      train_loop(ngram_model, optimizer, criterion, scheduler, train_ngram_dataloader, valid_ngram_dataloader, epochs)

      torch.save(ngram_model.state_dict(), 'ffnn.pth')


    if model_type == 'LSTM':
        pass
    if model_type == 'FFNN_CLASSIFY':
      valid_bios = split_bios(valid_data, valid_words)
      valid_bios = clean_bios(valid_bios, valid_words)
      valid_ngrams_data = create_ngrams(valid_bios, valid_words, window)

      valid_ngram_dataset = NgramDataset(valid_ngrams_data)
      valid_ngram_dataloader = DataLoader(valid_ngram_dataset, batch_size=1)

      test_bios = split_bios(test_data, test_words)
      test_bios = clean_bios(test_bios, test_words)
      test_ngrams_data = create_ngrams(test_bios, test_words, window)

      test_ngram_dataset = NgramDataset(test_ngrams_data)
      test_ngram_dataloader = DataLoader(test_ngram_dataset, batch_size=1)

      ngram_model = FFNN(train_vocab, train_words, d_model, d_hidden, dropout, window).to(device)
      ngram_model.load_state_dict(torch.load('ffnn.pth', map_location=torch.device('cpu')))
      ngram_model.eval()
        
      create_histogram(ngram_model, valid_ngram_dataloader, window, valid_words)
      threshold = 4500
      ffnn_test_model(ngram_model, test_ngram_dataloader, test_words, threshold)
    if model_type == 'LSTM_CLASSIFY':
        pass


In [ ]:
main(args)

<ipython-input-4-22eeab7ab3c1>:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(bio[stop]).to(device)


Epoch: 1 /////////////////////////////////////
Train Accuracy: 0.0635224516390775
Train Loss: 7.459181944755837
Train Perplexity: 1735.727555374415

Valid Accuracy: 0.0030896048326201505
Valid Loss: 9.189187093072714
Valid Perplexity: 9790.688824628192

Epoch: 2 /////////////////////////////////////
Train Accuracy: 0.09610456807765876
Train Loss: 6.889013646378257
Train Perplexity: 981.4328997512579

Valid Accuracy: 0.03781042872724307
Valid Loss: 9.372737751179908
Valid Perplexity: 11763.275965798688

Epoch: 3 /////////////////////////////////////
Train Accuracy: 0.11000974610838678
Train Loss: 6.754663227243297
Train Perplexity: 858.0507331139299

Valid Accuracy: 0.04037461196409099
Valid Loss: 9.394834029263464
Valid Perplexity: 12026.093534550402

Epoch: 4 /////////////////////////////////////


KeyboardInterrupt: ignored